In [12]:
import time
import numpy as np
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [18]:
sc = SparkContext.getOrCreate()
ssc = StreamingContext(sc, 1)
ssc.checkpoint("./checkpoint")

all_data = ssc.socketTextStream("10.128.0.4", 9999)


## For 10 day average
prices_10_day = all_data.window(10, 1).map(lambda line : float(line.split(' ')[1][1:]))
date_10_day = all_data.map(lambda line : str(line.split(' ')[0]))
sum_10_days = prices_10_day.reduce(lambda x, y: x + y)
cnt_10_day = all_data.countByWindow(10, 1)

date_for_10 = date_10_day.map(lambda date: [1, date])
sum_for_10 = sum_10_days.map(lambda val: [1, val])
cnt_for_10 = cnt_10_day.map(lambda val: [1, val])

joined_stream_10 = sum_for_10.join(cnt_for_10).join(date_for_10)
refined_stream_10 = joined_stream_10.map(lambda values: [values[1][0][0], values[1][0][1], values[1][1]])

avg_10_days = refined_stream_10.map(lambda triplet: [triplet[0] / triplet[1], triplet[2]])

## For 40 day average
prices_40_day = all_data.window(40, 1).map(lambda line : float(line.split(' ')[1][1:]))
date_40_day = all_data.map(lambda line : str(line.split(' ')[0]))
sum_40_days = prices_40_day.reduce(lambda x, y: x + y)
cnt_40_day = all_data.countByWindow(40, 1)

date_for_40 = date_40_day.map(lambda date: [1, date])
sum_for_40 = sum_40_days.map(lambda val: [1, val])
cnt_for_40 = cnt_40_day.map(lambda val: [1, val])

joined_stream_40 = sum_for_40.join(cnt_for_40).join(date_for_40)
refined_stream_40 = joined_stream_40.map(lambda values: [values[1][0][0], values[1][0][1], values[1][1]])

avg_40_days = refined_stream_40.map(lambda triplet: [triplet[0] / triplet[1], triplet[2]])



avg_10_temp = avg_10_days.map(lambda pair: [1, pair])
avg_40_temp = avg_40_days.map(lambda pair: [1, pair])

combined_averages = avg_10_temp.join(avg_40_temp)
finalized_combined_averages = combined_averages.map(lambda triplet: [[triplet[1][0][0], triplet[1][0][1]], [triplet[1][1][0], triplet[1][1][1]]])



result = finalized_combined_averages.map(lambda pairs: "({} BUY AAPL)".format(pairs[0][1]) if pairs[0][0] > pairs[1][0] else "({} SELL AAPL)".format(pairs[1][1]))
result.pprint()

22/11/17 00:57:13 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/11/17 00:57:13 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/11/17 00:57:13 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/11/17 00:57:13 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [19]:
ssc.start()

22/11/17 00:57:17 WARN org.apache.spark.streaming.StreamingContext: Dynamic Allocation is enabled for this application. Enabling Dynamic allocation for Spark Streaming applications can cause data loss if Write Ahead Log is not enabled for non-replayable sources. See the programming guide for details on how to enable the Write Ahead Log.


-------------------------------------------
Time: 2022-11-17 00:57:23
-------------------------------------------
(09/07/2021 SELL AAPL)
(09/08/2021 SELL AAPL)
(09/09/2021 SELL AAPL)
(09/10/2021 SELL AAPL)
(09/13/2021 SELL AAPL)
(09/14/2021 SELL AAPL)
(09/15/2021 SELL AAPL)
(09/16/2021 SELL AAPL)
(09/17/2021 SELL AAPL)
(09/20/2021 SELL AAPL)
...



-------------------------------------------
Time: 2022-11-17 00:57:24
-------------------------------------------
(02/15/2022 SELL AAPL)



-------------------------------------------
Time: 2022-11-17 00:57:25
-------------------------------------------
(02/16/2022 SELL AAPL)



-------------------------------------------
Time: 2022-11-17 00:57:26
-------------------------------------------
(02/17/2022 SELL AAPL)



...

-------------------------------------------
Time: 2022-11-17 01:05:29
-------------------------------------------



-------------------------------------------
Time: 2022-11-17 01:05:30
-------------------------------------------



-------------------------------------------
Time: 2022-11-17 01:05:31
-------------------------------------------



-------------------------------------------
Time: 2022-11-17 01:05:32
-------------------------------------------



22/11/17 01:05:33 ERROR org.apache.spark.rpc.netty.Inbox: An error happened while processing message in the inbox for CoarseGrainedScheduler
java.lang.StackOverflowError
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1841)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1534)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.writeArray(ObjectOutputStream.java:1378)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1174)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.ObjectOutputStream.defaultWriteObject(ObjectOutputStream.java:441)
	at org.apache.spark.rdd.Partitione

In [17]:
ssc.stop()

22/11/17 00:55:30 ERROR org.apache.spark.streaming.scheduler.ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
22/11/17 00:55:31 WARN org.apache.spark.streaming.util.BatchedWriteAheadLog: BatchedWriteAheadLog Writer queue interrupted.
22/11/17 00:55:31 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_1421_0 !
22/11/17 00:55:31 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_2470_0 !
22/11/17 00:55:31 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_2144_0 !
22/11/17 00:55:31 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_1208_0 !
22/11/17 00:55:31 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_1820_0 !
22/11/17 00:55:31 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_883_0 !
22/11/17 00:55:31 WARN or